In [1]:
# pip install nuscenes-devkit > /dev/null
# !wget -N https://www.nuscenes.org/data/v1.0-mini.tgz
# !mkdir -p data
# !tar -xzf v1.0-mini.tgz -C data

In [2]:
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='data', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.449 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

In [4]:
scene = nusc.scene[0]
scene

{'token': 'cc8c0bf57f984915a77078b10eb33198',
 'log_token': '7e25a2c8ea1f41c5b0da1e69ecfa71a2',
 'nbr_samples': 39,
 'first_sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'last_sample_token': 'ed5fc18c31904f96a8f0dbb99ff069c0',
 'name': 'scene-0061',
 'description': 'Parked truck, construction, intersection, turn left, following a van'}

In [5]:
nusc.get('sample', scene['first_sample_token'])

{'token': 'ca9a282c9e77460f8360f564131a8af5',
 'timestamp': 1532402927647951,
 'prev': '',
 'next': '39586f9d59004284a7114a68825e8eec',
 'scene_token': 'cc8c0bf57f984915a77078b10eb33198',
 'data': {'RADAR_FRONT': '37091c75b9704e0daa829ba56dfa0906',
  'RADAR_FRONT_LEFT': '11946c1461d14016a322916157da3c7d',
  'RADAR_FRONT_RIGHT': '491209956ee3435a9ec173dad3aaf58b',
  'RADAR_BACK_LEFT': '312aa38d0e3e4f01b3124c523e6f9776',
  'RADAR_BACK_RIGHT': '07b30d5eb6104e79be58eadf94382bc1',
  'LIDAR_TOP': '9d9bf11fb0e144c8b446d54a8a00184f',
  'CAM_FRONT': 'e3d495d4ac534d54b321f50006683844',
  'CAM_FRONT_RIGHT': 'aac7867ebf4f446395d29fbd60b63b3b',
  'CAM_BACK_RIGHT': '79dbb4460a6b40f49f9c150cb118247e',
  'CAM_BACK': '03bea5763f0f4722933508d5999c5fd8',
  'CAM_BACK_LEFT': '43893a033f9c46d4a51b5e08a67a1eb7',
  'CAM_FRONT_LEFT': 'fe5422747a7d4268a4b07fc396707b23'},
 'anns': ['ef63a697930c4b20a6b9791f423351da',
  '6b89da9bf1f84fd6a5fbe1c3b236f809',
  '924ee6ac1fed440a9d9e3720aac635a0',
  '91e3608f55174a319

In [6]:
nusc.get('sample_data', nusc.get('sample', scene['first_sample_token'])['data']['LIDAR_TOP'])

{'token': '9d9bf11fb0e144c8b446d54a8a00184f',
 'sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'ego_pose_token': '9d9bf11fb0e144c8b446d54a8a00184f',
 'calibrated_sensor_token': 'a183049901c24361a6b0b11b8013137c',
 'timestamp': 1532402927647951,
 'fileformat': 'pcd',
 'is_key_frame': True,
 'height': 0,
 'width': 0,
 'filename': 'samples/LIDAR_TOP/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402927647951.pcd.bin',
 'prev': '',
 'next': '0cedf1d2d652468d92d23491136b5d15',
 'sensor_modality': 'lidar',
 'channel': 'LIDAR_TOP'}

In [10]:
import rosbag
import rospy
import os
import math
import random
from pprint import pprint
from std_msgs.msg import ColorRGBA
from sensor_msgs.msg import PointCloud2, PointField
from tf2_msgs.msg import TFMessage
from geometry_msgs.msg import Pose, Transform, TransformStamped

from visualization_msgs.msg import Marker, MarkerArray

def get_transform(data):
    t = Transform()
    t.translation.x = data['translation'][0]
    t.translation.y = data['translation'][1]
    t.translation.z = data['translation'][2]
    
    t.rotation.w = data['rotation'][0]
    t.rotation.x = data['rotation'][1]
    t.rotation.y = data['rotation'][2]
    t.rotation.z = data['rotation'][3]
    
    return t

def get_pose(data):
    p = Pose()
    p.position.x = data['translation'][0]
    p.position.y = data['translation'][1]
    p.position.z = data['translation'][2]
    
    p.orientation.w = data['rotation'][0]
    p.orientation.x = data['rotation'][1]
    p.orientation.y = data['rotation'][2]
    p.orientation.z = data['rotation'][3]
    
    return p

def get_time(data):
    t = rospy.Time()
    t.secs, msecs = divmod(data['timestamp'], 1_000_000)
    t.nsecs = msecs * 1000
    
    return t

def random_color():
    c = ColorRGBA()
    c.r = random.uniform(0.5, 1)
    c.g = random.uniform(0.5, 1)
    c.b = random.uniform(0.5, 1)
    c.a = 0.5
    
    return c

first_sample = nusc.get('sample', scene['first_sample_token'])
sample_lidar = nusc.get('sample_data', first_sample['data']['LIDAR_TOP'])
ann_colors = {}
published_samples = set()

bag = rosbag.Bag('out.bag', 'w')

try:
    while sample_lidar is not None:
        print('sample_lidar', sample_lidar['token'])
            
        # create point cloud message
        pc_filename = 'data/' + sample_lidar['filename']
        pc_filesize = os.stat(pc_filename).st_size

        with open(pc_filename, 'rb') as pc_file:
            pc = PointCloud2()
            pc.header.frame_id = 'lidar'
            pc.header.stamp = get_time(sample_lidar)

            pc.fields = [
                PointField(name='x', offset=0, datatype=PointField.FLOAT32, count=1),
                PointField(name='y', offset=4, datatype=PointField.FLOAT32, count=1),
                PointField(name='z', offset=8, datatype=PointField.FLOAT32, count=1),
                PointField(name='intensity', offset=12, datatype=PointField.FLOAT32, count=1),
                PointField(name='ring', offset=16, datatype=PointField.FLOAT32, count=1),
            ]

            pc.is_bigendian = False
            pc.is_dense = True
            pc.point_step = len(pc.fields) * 4 # 4 bytes per field
            pc.row_step = pc_filesize
            pc.width = round(pc_filesize / pc.point_step)
            pc.height = 1 # unordered
            pc.data = pc_file.read()

            bag.write('/lidar', pc, pc.header.stamp)

        tf_array = TFMessage()
        
        # create ego transform
        ego_pose = nusc.get('ego_pose', sample_lidar['ego_pose_token'])
        ego_tf = TransformStamped()
        ego_tf.header.frame_id = 'world'
        ego_tf.header.stamp = get_time(ego_pose)
        ego_tf.child_frame_id = 'ego'
        ego_tf.transform = get_transform(ego_pose)
        tf_array.transforms.append(ego_tf)
        
        # create lidar transform
        lidar_tf = TransformStamped()
        lidar_tf.header.frame_id = 'ego'
        lidar_tf.header.stamp = get_time(sample_lidar)
        lidar_tf.child_frame_id = 'lidar'
        lidar_tf.transform = get_transform(
            nusc.get('calibrated_sensor', sample_lidar['calibrated_sensor_token']))
        tf_array.transforms.append(lidar_tf)
        
        bag.write('/tf', tf_array, lidar_tf.header.stamp)

        # create bounding boxes
        sample = nusc.get('sample', sample_lidar['sample_token'])
        if sample['token'] not in published_samples:
            print('sample', sample['token'])

            marker_array = MarkerArray()
            i = 0
            for ann_id in sample['anns']:
                ann = nusc.get('sample_annotation', ann_id)

                marker = Marker()
                marker.header.frame_id = 'world'
                marker.header.stamp = get_time(sample_lidar)
    #                 marker.ns = 'foo'
                marker.id = i
                marker.type = Marker.CUBE
                marker.pose = get_pose(ann)

                marker.scale.x = ann['size'][1]
                marker.scale.y = ann['size'][0]
                marker.scale.z = ann['size'][2]

                marker.color = ann_colors.get(ann['instance_token'])
                if marker.color is None:
                    marker.color = random_color();
                    ann_colors[ann['instance_token']] = marker.color

                marker_array.markers.append(marker)
                i+=1

                bag.write('/annotations', marker_array, get_time(sample_lidar))
            published_samples.add(sample['token'])

        print()
        sample_lidar = nusc.get('sample_data', sample_lidar['next']) if sample_lidar.get('next') != '' else None
        
    print('\nsuccess')
finally:
    bag.close()


sample_lidar 9d9bf11fb0e144c8b446d54a8a00184f
sample ca9a282c9e77460f8360f564131a8af5

sample_lidar 0cedf1d2d652468d92d23491136b5d15
sample 39586f9d59004284a7114a68825e8eec

sample_lidar 0865d9cca7f647c3a2d37f9ba7a2d7f4

sample_lidar 5eb7f6b2a4f24920adeea7460397b077

sample_lidar 2c7f800e64964d0bb9d07434d4e5fd46

sample_lidar 882d773341df4a319ac1247057ba576d

sample_lidar 0abbcec5738f46908b6db263ffbad3ae

sample_lidar 97e72e8dd2ac4d07bc0d863f16f2288a

sample_lidar f9e41eadc16247e58dc3b9119c607923

sample_lidar 6c68f70d05d74994817f11131fb21f3d

sample_lidar 4f792c8da81e4cb7aca1790654da1c27

sample_lidar 38f3e46f0845441295e639bc5a4e24ca
sample 356d81f38dd9473ba590f39e266f54e5

sample_lidar 94368c083c674f7a84c7273854bfc7ac

sample_lidar 024b8e9233e4498abe30708c476b3bae

sample_lidar 875d83640a1e46c789fcb459c3b12a7f

sample_lidar 224c8d75585d4bc284ef13ddd01975a6

sample_lidar 67f8fede7d9f44d2966cc8a2cf283c6c

sample_lidar 3d7dec968f6f42dc95c2f14d286f48fc

sample_lidar e40ff2a839cc41c98f31a